In [16]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
import pandas as pd
import pickle

In [8]:
train_file = "../data/train.pkl"
fr = open(train_file, 'rb')
train_data = pickle.load(fr)
fr.close()
users = list(train_data.keys())
num_users = len(users)

In [12]:
list_for_df = []
for user in users:
    for item in train_data[user]:
        list_for_df.append({
            "user": user,
            "item": item,
            "rating": 1
        })
data_df = pd.DataFrame(list_for_df)

In [14]:
data_df.head(5)

,user,item,rating
0,0,18,1
1,0,243,1
2,0,7706,1
3,0,2229,1
4,0,1037,1


In [18]:
reader = Reader(rating_scale=(1, 1))
train_dataset = Dataset.load_from_df(data_df[['user', 'item', 'rating']], reader)

In [19]:
# Use the famous SVD algorithm.
algo = SVD()
algo.fit(train_dataset)
# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
MAE (testset)     0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  
Fit time          3.74    3.90    3.88    3.94    3.99    3.89    0.08    
Test time         0.70    0.70    0.53    0.75    0.52    0.64    0.09    


{'test_rmse': array([0., 0., 0., 0., 0.]),
 'test_mae': array([0., 0., 0., 0., 0.]),
 'fit_time': (3.7398200035095215,
  3.895585298538208,
  3.8763926029205322,
  3.9394872188568115,
  3.987457752227783),
 'test_time': (0.696469783782959,
  0.6988294124603271,
  0.5335800647735596,
  0.7511825561523438,
  0.5192813873291016)}

In [20]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls